<a href="https://colab.research.google.com/github/AaronM44/DAOTW-Assignment-1/blob/main/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAOTW - Assignment 1

## Packages

### Install Packages

In [19]:
install.packages("patchwork")
install.packages("ggfortify")
install.packages("tidymodels")

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘future.apply’, ‘numDeriv’, ‘progressr’, ‘SQUAREM’, ‘Rcpp’, ‘lava’, ‘listenv’, ‘parallelly’, ‘prodlim’, ‘future’, ‘warp’, ‘iterators’, ‘lhs’, ‘DiceDesign’, ‘globals’, ‘clock’, ‘gower’, ‘ipred’, ‘timeDate’, ‘furrr’, ‘slider’, ‘foreach’, ‘GPfit’, ‘modelenv’, ‘conflicted’, ‘dials’, ‘hardhat’, ‘infer’, ‘modeldata’, ‘parsnip’, ‘recipes’, ‘rsample’, ‘tune’, ‘workflows’, ‘workflowsets’, ‘yardstick’




### Import Packages

In [20]:
library(tidyverse)
library(broom)
library(patchwork)
library(scales)
library(ggfortify)
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.0.0 ──

✔ dials        1.1.0     ✔ rsample      1.1.1
✔ infer        1.0.4     ✔ tune         1.0.1
✔ modeldata    1.0.1     ✔ workflows    1.1.2
✔ parsnip      1.0.3     ✔ workflowsets 1.0.0
✔ recipes      1.0.3     ✔ yardstick    1.1.0

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filter()   masks stats::filter()
✖ recipes::fixed()  masks stringr::fixed()
✖ dplyr::lag()      masks stats::lag()
✖ yardstick::spec() masks readr::spec()
✖ recipes::step()   masks stats::step()
• Search for functions across packages at https://www.tidymodels.org/find/



## Import Data

In [10]:
weather <- read.csv("https://raw.githubusercontent.com/AaronM44/DAOTW-Assignment-1/main/data/weather.csv")

In [11]:
collated <- read.csv("https://raw.githubusercontent.com/AaronM44/DAOTW-Assignment-1/main/data/collated_collision_data.csv")

In [12]:
contributing_factors <- read.csv("https://raw.githubusercontent.com/AaronM44/DAOTW-Assignment-1/main/data/contributing_factors.csv")

### SQL Used To Create Datasets

#### Weather

#### Collated Collisions

#### Contributing Factors

## Dataset Transformations

### Contributing Factors

### Collisions

Remove missing values as per the key in the below section (Field Descriptions)

In [36]:
collated_clean <- collated %>%
  mutate(across(c(dewp, slp, max, min), na_if, 9999.9))

collated_clean <- collated_clean %>%
  mutate(across(c(visib, wdsp, mxpsd, gust, sndp), na_if, 999.9))

collated_clean <- collated_clean %>%
  mutate(prcp = na_if(prcp, 99.99))

## Dataset Summaries

### Collisions

#### Field Descriptions

[Information on these descriptions was found on the noaa site](https://www.ncei.noaa.gov/data/global-summary-of-the-day/doc/readme.txt)


*   DEWP - Mean dew point for the day in degrees Fahrenheit to tenths. Missing = 9999.9
*   SLP - Mean sea level pressure for the day in millibars to tenths. Missing = 9999.9
* VISIB - Mean visibility for the day in miles to tenths. Missing = 999.9
* WDSP - Mean wind speed for the day in knots to tenths.  Missing = 999.9
* MXSPD - Maximum sustained wind speed reported for the day in knots to tenths. Missing = 999.9
* GUST - Maximum wind gust reported for the day in knots to tenths.  Missing = 999.9
* MAX - Maximum temperature reported during the day in Fahrenheit to tenths. Missing = 9999.9
* PRCP - Total precipitation (rain and/or melted snow) reported during the day in inches
 and hundredths; will usually not end with the midnight observation (i.e. may include
latter part of previous day). “0” indicates no measurable precipitation (includes a trace).   
Missing = 99.99
* SNDP - Snow depth in inches to tenths. It is the last report for the day if reported more than once. Missing = 999.9



In [21]:
head(collated)

,day,year,mo,da,collision_date,temp,dewp,slp,visib,wdsp,mxpsd,gust,max,min,prcp,sndp,fog,num_collisions
,<int>,<int>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>
1,2,2018,1,2,2018-01-02,14.7,2.0,1024.9,10.0,12.9,20.0,31.1,18,5.0,0.00,999.9,0,654
2,1,2018,1,1,2018-01-01,9.5,-0.5,1022.2,10.0,15.0,18.1,25.1,21,5.0,0.00,999.9,0,453
3,5,2014,1,24,2014-01-24,14.8,-0.5,1022.4,10.0,11.8,15.9,22.9,23,9.0,0.00,999.9,0,705
4,4,2015,1,8,2015-01-08,13.1,-2.4,1022.7,9.9,14.8,21.0,29.9,27,6.1,0.00,999.9,0,573
5,6,2018,1,6,2018-01-06,13.2,-0.2,1013.5,10.0,22.4,28.0,39.0,27,10.9,0.00,999.9,0,599
6,4,2014,1,30,2014-01-30,22.0,5.8,1027.2,10.0,8.1,15.9,24.1,27,12.9,0.01,999.9,0,657


In [37]:
summary(collated_clean)

      day             year            mo               da       
 Min.   :1.000   Min.   :2012   Min.   : 1.000   Min.   : 1.00  
 1st Qu.:2.000   1st Qu.:2014   1st Qu.: 4.000   1st Qu.: 8.00  
 Median :4.000   Median :2016   Median : 7.000   Median :16.00  
 Mean   :3.999   Mean   :2016   Mean   : 6.699   Mean   :15.74  
 3rd Qu.:6.000   3rd Qu.:2018   3rd Qu.:10.000   3rd Qu.:23.00  
 Max.   :7.000   Max.   :2020   Max.   :12.000   Max.   :31.00  
                                                                
 collision_date          temp            dewp            slp      
 Length:3106        Min.   : 5.80   Min.   :-6.70   Min.   : 989  
 Class :character   1st Qu.:41.20   1st Qu.:32.60   1st Qu.:1012  
 Mode  :character   Median :52.50   Median :45.70   Median :1017  
                    Mean   :52.25   Mean   :44.67   Mean   :1017  
                    3rd Qu.:64.70   3rd Qu.:59.10   3rd Qu.:1022  
                    Max.   :86.70   Max.   :74.10   Max.   :1044  
           

#### Num. Collisions
Min: 94  
Max: 1161  
Mean: 561.8  
Median: 580